In [13]:
import itertools
import pandas as pd
import re
from log_class import log_recorder

LOG_DIR = "./test_data2/"
report_csv = "report.csv"
LOG_file = "LOG"


# from log_class import log_recorder
COMPACTION_LOG_HEAD = "/compaction/compaction_job.cc:755"
FLUSH_LOG_BEGIN = "flush_started"
FLUSH_LOG_END = "flush_finished"
FLUSH_FILE_CREATEION = "table_file_creation"

def load_log_and_qps(log_file, ground_truth_csv):
    # load the data
    return log_recorder(log_file,ground_truth_csv)

data_set = load_log_and_qps(LOG_DIR+LOG_file, LOG_DIR+report_csv)

# Cut the timeline into pieces according to the time slice size

In [14]:
ms_to_sec  = 1000000
time_slice = 100000 # 100 miusec, 100,000ms
switch_ratio = ms_to_sec/time_slice

real_time_speed=data_set.qps_df
flush_jobs = data_set.flush_df
# op_type, 0 for flush, 1 for compaction; 
# triggered or not, 0 for not triggered, 1 for running
# input size, how many bytes have been read from disk
# total_operation_bytes
# job_id, i don't kown whether it can be used or not.
input_tuple=[0,0,12345,7] 

# then we put all the operations into a series of input tupler stack
#        -----
#      ----
# --- -- 
# -- -------
# 1 2 3 4 5 6 7 8 9 10



# create the bucket first

# print(int(flush_jobs[flush_jobs["job"]==11902]["start_time"]/time_slice))


In [15]:

bucket = []
for i in range(int(real_time_speed.tail(1)["microsecs_elapsed"] * switch_ratio)):
    bucket.append([])
# then we use a bucket sort idea to count down the rest things
for flush_job in data_set.flush_df.iloc():
    # indices = (int(flush_job["start_time"]/time_slice), flush_job["end_time"]/time_slice)
    start_index = int(flush_job["start_time"]/time_slice)
    end_index=int(flush_job["end_time"]/time_slice) + 1
    payload = round(flush_job["flush_size"] / (1024*1024) ,2) # change to MB will be easier to calculate
    job_id = flush_job["job"]

    if start_index >= len(bucket)-10 or end_index >= len(bucket)-5: # the tail part is not accurant
        break
    for bucket_element in bucket[start_index:end_index]:
        bucket_element.append([0,0,payload,job_id])


for compaction_job in data_set.compaction_df.iloc():
    start_index = int(compaction_job["start_time"]/time_slice)
    end_index=int(compaction_job["end_time"]/time_slice) + 1
    input_size = round(compaction_job["input_data_size"] / (1024*1024) ,2) # change to MB will be easier to calculate
    output_size = round(compaction_job["total_output_size"] / (1024*1024) ,2)
    job_id = compaction_job["job"]
    if start_index >= len(bucket)-10 or end_index >= len(bucket)-5: # the tail part is not accurant
        break
    compaction_tuple=[1,input_size,output_size,job_id]
    for bucket_element in bucket[start_index:end_index]:
        bucket_element.append(compaction_tuple)

result_tensor = real_time_speed["interval_qps"]
max_length = max([len(x) for x in bucket])

In [16]:

empty_tuple = [0,0,0,0]
for element in bucket:
    while len(element) < max_length:
        element.append(empty_tuple)
import torch
import numpy as np 

In [17]:
input_tensor = torch.tensor(bucket)
qps_list = list(data_set.qps_df["interval_qps"]) 
while len(qps_list) < input_tensor.size(0):
    qps_list.append(0)

output_tensor = torch.tensor(qps_list).reshape(-1,1)

print(input_tensor.size(),output_tensor.size())

torch.Size([12000, 24, 4]) torch.Size([12000, 1])


In [18]:
flatten_input = []
for element in bucket:
    temp_list = []
    for xx in element:
        for x in xx:
            temp_list.append(x)
    flatten_input.append(temp_list)

flatten_input_tensor = torch.tensor(flatten_input)
# # print(flatten_input)
print(flatten_input_tensor.size())


torch.Size([12000, 96])


In [19]:
time_idx = set(data_set.qps_df["microsecs_elapsed"])
aggregated_qps = []
for time_id in time_idx:
    aggregated_qps.append(sum(data_set.qps_df["interval_qps"][time_id == data_set.qps_df["microsecs_elapsed"]]))

aggregate_output_tensor = torch.tensor(aggregated_qps).reshape(-1,1)

In [55]:
import matplotlib.pyplot as plt
# plt.plot(aggregated_qps)
from torch.autograd import Variable
X = Variable(flatten_input_tensor.double())
Y = Variable(aggregate_output_tensor.double())
import torch.nn.functional as F



In [43]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x

In [56]:
net = Net(X.size(1),100,Y.size(1))
net = net.double()
optimizer = torch.optim.SGD(net.parameters(),lr=0.2)
loss_func = torch.nn.MSELoss()
for t in range(200):
    prediction = net(X)
    # print(prediction)
    loss = loss_func(prediction, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #2 'mat1' in call to _th_addmm